![header](header10.2.png)

# Inheritance, composition, and magic methods

Now that we have played around making some classes, we can get deeper into object-oriented programming.


## Inheritance

Classes can beget classes, via inheritance. The language of "parent" and "child" classes is commonly used. Essentially, the child class has the attributes and methods of its parent, possibly with some modification, plus its own unique methods and attributes.

An example: in the footnote to the Data Structures video, we ran the following code:

In [8]:
from collections import Counter

a_list = ['A', 'B', 'B', 'C']
count = Counter(a_list)
print(count)

Counter({'B': 2, 'A': 1, 'C': 1})


Now we can unpack what is really going on. <code>Counter</code> is a class provided by the collections module. It provides us with a dictionary containing the number of occurrences of each item in the sequence, but it also comes with extra features: 

In [3]:
print(count + count)

Counter({'B': 4, 'A': 2, 'C': 2})


In [5]:
print(count.most_common(1))

[('B', 2)]


How can a dictionary do this? The answer, as you might have guessed, is that the <code>Counter</code> class is a child of the <code>dict</code> class. Thus, it has the properties of a dictionary, plus the additional features of <code>Counter</code>. Clearly, this is a powerful tool to have in our kit. I would encourage you, after reading this article, to dig up the source code of the Counter so you can see how it works.

The basic syntax of inheritance is easy enough.

In [2]:
class MyParentClass():
    def __init__(self):
        print("Parent class made")
        
class MyChildClass(MyParentClass):
    pass

child = MyChildClass()

Parent class made


Notice what happened here; because we did not supply the child class with an <code>\__init\__()</code> method, it defaulted to using the one provided by its parent. However:

In [6]:
class AnotherChildClass(MyParentClass):
    def __init__(self):
        print("A new __init__() method")
        
second_child = AnotherChildClass()

A new __init__() method


We can run both; simply call the parent's method from inside the child's method:

In [3]:
class AFinalChildClass(MyParentClass):
    def __init__(self):
        MyParentClass.__init__(self) # call init of parent
        print("And now anything extra from the child") # new stuff!
        
youngest_child = AFinalChildClass()

Parent class made
And now anything extra from the child


Of course, just like in the real world, these child classes can go on to have children of their own. Therefore a well designed inheritance structure for your program can cut down on your programming time significantly, since methods of parents can be reused in their children.

## Introspection

Introspection in Python boils down to the ability of programs to examine the structure of objects. We have already met the <code>dir()</code> function as an example, which lists the available methods of an object.

We'll now meet some more introspection tools.

## isinstance

Imagine you're writing a function or method, and you wish for the program to exhibit a different behaviour depending on the type or class of the argument that function receives. The function <code>isinstance()</code> takes as its first argument an object, and as a second argument a class. It returns a boolean.

In [1]:
print(isinstance([1, 2, 3], list))

True


In [4]:
print(isinstance(youngest_child, AFinalChildClass))

True


In [7]:
print(isinstance(youngest_child, AnotherChildClass))

False


Very importantly, <code>isinstance</code> considers instances of child classes to be instances of their parents. This means, for instance, that an if-statement that uses <code>isinstance()</code> to look for dictionaries would consider a counter object to be such an instance, as it is a child of the dictionary class:

In [10]:
print(isinstance(count, Counter))
print(isinstance(count, dict))

True
True


### hasattr

A similar function exists for checking whether an object possesses a particular attribute or method:

In [16]:
print(hasattr([1,2,3], 'sort'))

True


This will become much more useful when we meet the magic methods.

## Composition

As well as inheriting, we can create new classes by combining other classes. This is called composition. The idea is that the composite class has the different component classes as its attributes, and these are structured in a way which allows them to "talk" to one another. Here is one way you could do that:

In [8]:
class ComponentA():
    def __init__(self):
        self.name = "A"
        self.owner = None
        
    def show_owner(self):
        print("My owner is: ", self.owner.name)
        


class ComponentB():
    def __init__(self):
        self.name = "B"
        self.owner = None

    def show_owner(self):
        print("My owner is: ", self.owner.name)
        

        
class Composite():
    def __init__(self):
        self.name = "A + B"
        self.partA = ComponentA()
        self.partA.owner = self
        self.partB = ComponentB()
        self.partB.owner = self
        
    def my_parts(self):
        print("Part {} says: ".format(self.partA.name))
        self.partA.show_owner()
        print("Part {} says: ".format(self.partB.name))
        self.partB.show_owner()

In [7]:
a_composite_class = Composite()
a_composite_class.my_parts()

Part A says: 
My owner is:  A + B
Part B says: 
My owner is:  A + B


This is quite subtle actually, so read it carefully. The components have been given an "owner" attribute (again, this isn't a keyword, it is just chosen by convention). Now from inside each component, we can use <code>self.owner</code> to refer to the composite class, and inside the composite class, we can refer directly to each part as <code>self.part</code>. The confusing line to ponder is <code>self.partA.owner = self</code>. Alternatively, we could have set our component classes to expect an "owner" as an argument of their <code>\__init\__()</code> method, in which case the same code might look like this:

In [4]:
class ComponentA():
    def __init__(self, owner=None):
        self.name = "A"
        self.owner = owner
        
    def show_owner(self):
        print("My owner is: ", self.owner.name)
        


class ComponentB():
    def __init__(self, owner=None):
        self.name = "B"
        self.owner = owner
        
    def show_owner(self):
        print("My owner is: ", self.owner.name)
        

        
class Composite():
    def __init__(self):
        self.name = "A + B"
        self.partA = ComponentA(owner=self)
        self.partB = ComponentB(owner=self)
        
    def my_parts(self):
        print("Part {} says: ".format(self.partA.name))
        self.partA.show_owner()
        print("Part {} says: ".format(self.partB.name))
        self.partB.show_owner()

In [5]:
a_composite_class = Composite()
a_composite_class.my_parts()

Part A says: 
My owner is:  A + B
Part B says: 
My owner is:  A + B


These are basically equivalent solutions.

## Magic methods

In the first section on object-oriented programming, we discussed how in Python everything is an object, and showed that many of these objects exhibit cool behaviours, such as <code>+</code> being used to join together two strings, or <code>list[a:b]</code> giving us a slice of a list. We can build these behaviours into our own classes using the so-called magic methods.

Magic methods are essentially methods that are called whenever class is used together with a certain bit of in-built Python syntax. We have already met the very import magic method <code>\__init\__()</code> -- this method is called whenever the class is instantiated by writing <code>ClassName()</code>. Magic methods are easily identified by the two underscores either side of the name.

For example, let's build a "set" that contains everything, just to spite Bertrand Russell.

In [33]:
class ContainsEverything(set):
    def __contains__(self, item):
        return True


In this case, we have not chosen method names arbitrarily: these method names are very special, and each one refers to a particular aspect of Python's structure.

In [34]:
superset = ContainsEverything()

In [35]:
print("Spam" in superset)

True


In [36]:
print(0 in superset)

True


In [37]:
print(superset in superset)

True


As we can easily see, this object has been designed so that whenever it is placed in the context of "in", then it returns <code>True</code>, regardless of what we actually put in the set or what we check for membership. There are many magic methods, far too many to list here. The following example will show us a good few more though, and hopefully give you the confidence to explore them further for yourself. 

## Extended example: a polynomial class

Let's recap some maths, and then build a class that makes it all really easy.

In mathematics, a polynomial is a symbolic expression where a symbol, say, $x$, can be added, subtracted, multiplied by a number, and raised to an integer power. For example

$$3 + x - 2x^2$$

is a polynomial. So is

$$x^3 + 36x^9.$$

In general, a polynomial has the form

$$a_0 + a_1x + a_2x^2 + \dots + a_n x^n$$

where $n$ is called the degree of the polynomial, and the $a_i$'s are called coefficients. Polynomials can be added together by adding together the cofficients of corresponding powers of $x$ (if the degree of the polynomials is different, we consider the smaller polynomial to have $0x^{n+1} + \dots + 0x^{\deg{p}}$ added to the end, where $p$ is the polynomial with the higher degree). Example:

$$\underbrace{(3 + x - 2x^2)}_\text{Smaller degree} + (2 - x + x^3) = (3 + x - 2x^2 + 0x^3) + (2 - x + x^3) \\= (2 + 3) + (1-1)x -2x^2 + (0+1)x^3 = 5 -2x^2 + x^3$$

A polynomial can be multiplied by a number by multiplying each coefficient by the number. A polynomial can also be multiplied by another polynomial by repeated use of the distributive law, that $(a + b)c = ac + bc$. Example:

$$(1 - 2x + 3x^2)(2 + x) = 1(2 +x) -2x(2+x) + 3x^2(2+x) \\= (2 + x) - (4x + 2x^2) + (6x^2 + 3x^3) \\ =2 + (x - 4x) +(- 2x^2 + 6x^2) + 3x^3\\ = 2 - 3x + 4x^2 + 3x^3$$

A polynomial can be <i>evaluated</i> at a value by replacing all instances of $x$ with a number, and calculating the result. For instance, to evaluate $2 + x^2$ at $x = 3$, we calculate

$$2 + (3)^2 = 11.$$

The <i>derivative</i> of a polynomial is obtained by replacing each $a_k x^k$ with $ka_kx^{k-1}$. The $a_0$ term vanishes.

Polynomials are important in all areas of mathematics, and science. We'll make a Python script that can do all of these basic polynomial calculations in a natural way, using a <code>Polynomial</code> class.

## The example

The basic plan is that our polynomial will be a subclass of <code>list</code>. This is because we can exploit the fact that the only real information needed to characterize a polynomial is the coefficient of each term. These can be stored as a list, in which the indices of the list correspond precisely with the coefficient of that power of $x$ (for example, the <code>[3]</code>rd entry in a list will be the coefficient of $x^3$). We will exploit this fact as much as possible.

The only information we will specify to the polynomial is a sequence of coefficients, therefore. However, I'd like the <code>\__init\__()</code> method to be basically agnostic as to whether the sequence of terms is provided simply as separate arguments, or as a list/tuple. So we'll allow an arbitrary number of arguments, and then use our new introspection abilities to check what kind of argument is provided, and then modify the arguments if necessary:

Let's start with that now:

In [49]:
class Polynomial(list):
    '''
    Polynomial class, essentially a list of coefficients with
    a nice string representation, and correctly behaving addition,
    multiplication, etc.
    '''
    def __init__(self, *coeffs):
        from collections.abc import Sequence
        # The Sequence object can be used to check whether
        # an object is sequence-like.
        # it is an "abstract base class" -- you can't make objects from it
        # but other classes can inherit from it
        if isinstance(coeffs[0], Sequence):
            coeffs = coeffs[0]
        coeffs = list(coeffs)
        list.__init__(coeffs)
        

So all we've done so far is made sure the <code>coeffs</code> parameter refers to a sequence, and then passed it to the usual list <code>\__init\__()</code>. If we didn't include the first step, then when we call <code>Polynomial([1,2,3])</code>, the coeffs parameter would refer to <code>[[1,2,3]]</code> -- a list within a list.

I'd also like to be able to check the degree of my polynomial: the highest power of $x$ with a non-zero coefficient. Let's make a method which can do that for us. Figure out how this simple algorithm works!

In [50]:
class Polynomial(list):
    def __init__(self, *coeffs):
        from collections.abc import Sequence
        # The Sequence object can be used to check whether
        # an object is sequence-like.
        # it is an "abstract base class" -- you can't make objects from it
        # but other classes can inherit from it
        if isinstance(coeffs[0], Sequence):
            coeffs = coeffs[0]
        coeffs = list(coeffs)
        list.__init__(coeffs)
        self.degree = self.find_degree()
        
    def find_degree(self):
        '''
        Find the degree of the polynomial.
        This is not necessarily the length of the polynomial
        as a list, since there may be redundant 0 terms at the end
        '''
        for i, coefficient in enumerate(self):
            if coefficient != 0:
                degree = i
        return degree

Now, we would like the polynomial not to appear as a list to the user, but as a nicely formatted string. These are the criterion for the representation:

* Terms with coefficient 0 should be skipped and not displayed at all.
* The constant term $a_0$ should appear just as a number.
* If the coefficient is 1, don't display the coefficient ie. $x$ instead of $1x$.
* The power should not be displayed in the linear ($x$) term, ie. $x$ not $x^1$.

For this, we'll write a method that figures out how to display each term. Then we'll use the <code>\__str\__()</code> magic method to bring all the nicely formatted terms together and display them whenever the user writes <code>print(Polynomial)</code> or otherwise converts the polynomial to a string. A term of a polynomial is called a monomial:

In [52]:
class Polynomial(list):
    '''
    Polynomial class, essentially a list of coefficients with
    a nice string representation, and correctly behaving addition,
    multiplication, etc.
    '''
    def __init__(self, *coeffs):
        from collections.abc import Sequence
        if isinstance(coeffs[0], Sequence):
            coeffs = coeffs[0]
        list.__init__(self,coeffs)
        self.degree = self.find_degree()

    def find_degree(self):
        '''
        Find the degree of the polynomial.
        This is not necessarily the length of the polynomial
        as a list, since there may be redundant 0 terms at the end
        '''
        for i, coefficient in enumerate(self):
            if coefficient != 0:
                degree = i
        return degree

    def display_monomial(self, degree, coefficient):
        '''
        Correctly give a string representation of the given monomial
        '''
        if degree == 0:
            return str(coefficient)
        if degree == 1 and coefficient == 1:
            return self.symbol
        elif degree == 1 and coefficient != 1:
            return "{}{}".format(coefficient, "x")
        elif coefficient == 1:
            return "{}^{}".format("x", degree)
        else:
            return "{}{}^{}".format(coefficient, "x", degree)
        
    def __str__(self):
        monos = [self.display_monomial(k, coeff) for k, coeff in enumerate(self) if coeff != 0]
        return " + ".join(monos)

Now check it out!

In [55]:
simple_cubic = Polynomial((3, 7, 3, 1))
print(simple_cubic)

3 + 7x + 3x^2 + x^3


Looks good, eh? It's a list, but it displays differently when we print it. Neat.

Now, another fact about polynomials is that they can be evaluated at a number by replacing each instance of the indeterminate $x$ with that number. We will make it so performing this operation is like calling a function, so for a polynomial $p$, $p(A)$ evaluates the polynomial at the point $A$. To do this, we use the magic method <code>\__call\__()</code>. The actual algorithm will take advantage of the fact that list indices are powers. By using <code>enumerate()</code>, we get a pair whose first element is the power, and second is the coefficient. All we need to do is evaluate $a_k(A)^k$ for each $k$ in the polynomial, and add together the results:

In [56]:
class Polynomial(list):
    '''
    Polynomial class, essentially a list of coefficients with
    a nice string representation, and correctly behaving addition,
    multiplication, etc.
    '''
    def __init__(self, *coeffs):
        from collections.abc import Sequence
        if isinstance(coeffs[0], Sequence):
            coeffs = coeffs[0]
        list.__init__(self,coeffs)
        self.degree = self.find_degree()

    def find_degree(self):
        '''
        Find the degree of the polynomial.
        This is not necessarily the length of the polynomial
        as a list, since there may be redundant 0 terms at the end
        '''
        for i, coefficient in enumerate(self):
            if coefficient != 0:
                degree = i
        return degree

    def display_monomial(self, degree, coefficient):
        '''
        Correctly give a string representation of the given monomial
        '''
        if degree == 0:
            return str(coefficient)
        if degree == 1 and coefficient == 1:
            return self.symbol
        elif degree == 1 and coefficient != 1:
            return "{}{}".format(coefficient, "x")
        elif coefficient == 1:
            return "{}^{}".format("x", degree)
        else:
            return "{}{}^{}".format(coefficient, "x", degree)
        
    def __str__(self):
        monos = [self.display_monomial(k, coeff) for k, coeff in enumerate(self) if coeff != 0]
        return " + ".join(monos)
    
    def __call__(self, point):
        '''
        Evaluates the polynomial at the specified input
        '''
        return sum([co*point**i for i, co in enumerate(self)])

In [57]:
simple_cubic = Polynomial((3, 7, 3, 1))
simple_cubic(5)

238

Now let's deal with the arithmetic operations. To make the polynomials a little easier to work with, we'll also allow the polynomials to have a number added (which just adds to the constant ($a_0$) term), or be multiplied by a number, which multiplies each term by that number. We'll deal with this after, though.

So, let's start with adding. Recall that if the polynomials are of different lengths, then the shorter polynomial is lengthened first with 0-coefficient terms. So we'll have to do that, as well.

The <code>\__add\__()</code> method takes an additional argument alongside <code>self</code>: it is the thing to be added, of course. So <code>foo + bar</code> calls <code>foo.\__add\__(bar)</code>, essentially.

The strategy will be to sort the two summands by length, and then generate a new polynomial with 0s added to the shorter one, then return a polynomial forged by summing the corresponding terms. For this, we'll create a "match length" method, too. 

In [ ]:
    def match_length(self, p):
        '''
        Adding polynomials requires them to be same length
        This method returns the same polynomial with additional zeros added to the end
        to match the length of the polynomial p
        '''
        added_zeros = [0 for i in range(len(p) - len(self))]
        return Polynomial(list(self) + added_zeros)
    
    
    def __add__(self, p):
        polys = [self, p]
        polys.sort(key=len)
        matched_length = polys[0].match_length(polys[1])
        return Polynomial([co[0] + co[1] for co in zip(polys[1], matched_length)])

So let's break it down real quick. First we create a list with the summands, then we sort them using key=len, which sorts by length. Then we create a new polynomial by calling the <code>match_length</code> method on the shorter of the two polynomials.

Finally, we output a new polynomial. This list comprehension looks a little bit busy, but we can break it down. What the "loop" part <code>for co in zip(polys[1], matched_length)</code>, doing? Well, <code>zip()</code> takes two lists and creates a generator that generators corresponding pairs:

In [59]:
list1 = [1, 2, 3]
list2 = ["a", "b", "c"]
list3 = list(zip(list1, list2))
print(list3)

[(1, 'a'), (2, 'b'), (3, 'c')]


Then <code>co[0] + co[1]</code> is simply summing the pairs of corresponding coefficients.

Now, we also want to be able to add a number. To do this, we will first create a method to check whether an input is in fact a number. If it is a number, we'll turn it into a polynomial with 1 term, and then just add it using the addition algorithm we already defined just now.

In [ ]:
    def isnumber(self, p):
        from numbers import Number
        return isinstance(p, Number)
    
    def __add__(self, p):
        from numbers import Number
        # First, if we add a number rather than another polynomial,
        # we just return a polynomial whose constant (0th) term has
        # p added
        if self.isnumber(p):
            return self + Polynomial(p)

        polys = [self, p]
        polys.sort(key=len)
        matched_length = polys[0].match_length(polys[1])
        return Polynomial([co[0] + co[1] for co in zip(polys[1], matched_length)])

All together, we now have: 

In [60]:
class Polynomial(list):
    '''
    Polynomial class, essentially a list of coefficients with
    a nice string representation, and correctly behaving addition,
    multiplication, etc.
    '''
    def __init__(self, *coeffs):
        from collections.abc import Sequence
        if isinstance(coeffs[0], Sequence):
            coeffs = coeffs[0]
        list.__init__(self, coeffs)
        self.degree = self.find_degree()

    def find_degree(self):
        '''
        Find the degree of the polynomial.
        This is not necessarily the length of the polynomial
        as a list, since there may be redundant 0 terms at the end
        '''
        for i, coefficient in enumerate(self):
            if coefficient != 0:
                degree = i
        return degree

    def display_monomial(self, degree, coefficient):
        '''
        Correctly give a string representation of the given monomial
        '''
        if degree == 0:
            return str(coefficient)
        if degree == 1 and coefficient == 1:
            return self.symbol
        elif degree == 1 and coefficient != 1:
            return "{}{}".format(coefficient, "x")
        elif coefficient == 1:
            return "{}^{}".format("x", degree)
        else:
            return "{}{}^{}".format(coefficient, "x", degree)


    def __str__(self):
        monos = [self.display_monomial(k, coeff) for k, coeff in enumerate(self) if coeff != 0]
        return " + ".join(monos)


    def __call__(self, point):
        '''
        Evaluates the polynomial at the specified input
        '''
        return sum([co*point**i for i, co in enumerate(self)])


    def match_length(self, p):
        '''
        Adding polynomials requires them to be same length
        This method returns the same polynomial with additional zeros added to the end
        to match the length of the polynomial p
        '''
        added_zeros = [0 for i in range(len(p) - len(self))]
        return Polynomial(list(self) + added_zeros)

    def isnumber(self, p):
        from numbers import Number
        return isinstance(p, Number)
                    
                    
    def __add__(self, p):
        from numbers import Number
        # First, if we add a number rather than another polynomial,
        # we just return a polynomial whose constant (0th) term has
        # p added
        if self.isnumber(p):
            return self + Polynomial(p)

        polys = [self, p]
        polys.sort(key=len)
        matched_length = polys[0].match_length(polys[1])
        return Polynomial([co[0] + co[1] for co in zip(polys[1], matched_length)])

In [61]:
a_polynomial = Polynomial(43, -23, 2, 0, 4)

In [63]:
print(a_polynomial)
print(a_polynomial + a_polynomial) 
print(a_polynomial + 10)

43 + -23x + 2x^2 + 4x^4
86 + -46x + 4x^2 + 8x^4
53 + -23x + 2x^2 + 4x^4


Works like a charm. Now we have addition, defining subtraction is easy. We first define the negative of a polynomial, and then define subtraction to be adding the negative:

In [ ]:
    def __neg__(self):
        return Polynomial([-a for a in self])

    def __sub__(self, p):
        return self + (-p)

Now then, multplication. Actually, I'm not so happy with the multiplication algorithm here. It's very ugly, and in no way easy to understand. I'll explain it briefly, but I also encourage you to have a go at doing a better job than me. The algorithm is based on the fact that the coefficient of the $k$th term in a polynomial that is the product of two polynomials is the sum of products of coefficients in the multiplicands whose indices sum to $k$. So I use a generator to generator pairs of numbers that sum to $k$, and then multiply the corresponding terms using a nested list comprehension. Mathematically, if polynomials with coefficients $b_i$s and $c_j$s are multiplied, then

$$a_k = \sum_{i + j = k} b_ic_j,$$

which may be clearer if you are used to reading mathematical formulas.

Honestly, feel free to skip over this section.

In [ ]:
    def same_sum_pairs(self, i, j): 
        '''
        Generates tuples containing pairs of powers
        having the same sum, upto required degree
        '''
        from itertools import product
        for k in range(i + j - 1): # need to -1 here because polynomial degree = highest power =
            yield (pair for pair in product(range(i), range(j)) if pair[0] + pair[1] == k)
            
    def __mul__(self,p):
        # Multiplication by a number just multiplies each
        # term by the number
        if self.isnumber(p):
            return self * Polynomial(p)
        deg1, deg2 = len(self), len(p)
        terms = [[self[i]*p[j] for i, j in degree_pairs] # list comprehensions within list comprehensions
                for degree_pairs in self.same_sum_pairs(deg1, deg2)] # never go deeper!
        return Polynomial([sum(term) for term in terms])

There's one final thing we should account for. We have told the polynomial what to do if it is in the situation: <code>Polynomial + something</code> and <code>Polynomial * something</code>. But what about <code>something + Polynomial</code>? Usually this will work anyway because the <code>something</code> will itself be a polynomial, and so Python will use <code>something</code>'s addition method. But we have also allowed <code>something</code> to be a number, which doesn't no what to do if it meets our polynomial object. Thankfully this is easily fixed. The magic methods <code>\__radd\__()</code> and <code>\__rmul\__()</code> (I think the r stands for right) is for this situation, and in that case it's easy to just to swap the summands or multiplicands round:

In [ ]:
    def __radd__(self, p):
        return self + p # turns p + self into self + p, for example.

    def __rmul__(self, p):
        return self * p

So let's put all that together before we tackle the derivative operation.

In [64]:
class Polynomial(list):
    '''
    Polynomial class, essentially a list of coefficients with
    a nice string representation, and correctly behaving addition,
    multiplication, etc.
    '''
    def __init__(self, *coeffs):
        from collections.abc import Sequence
        if isinstance(coeffs[0], Sequence):
            coeffs = coeffs[0]
        list.__init__(self, coeffs)
        self.degree = self.find_degree()

    def find_degree(self):
        '''
        Find the degree of the polynomial.
        This is not necessarily the length of the polynomial
        as a list, since there may be redundant 0 terms at the end
        '''
        for i, coefficient in enumerate(self):
            if coefficient != 0:
                degree = i
        return degree

    def display_monomial(self, degree, coefficient):
        '''
        Correctly give a string representation of the given monomial
        '''
        if degree == 0:
            return str(coefficient)
        if degree == 1 and coefficient == 1:
            return self.symbol
        elif degree == 1 and coefficient != 1:
            return "{}{}".format(coefficient, "x")
        elif coefficient == 1:
            return "{}^{}".format("x", degree)
        else:
            return "{}{}^{}".format(coefficient, "x", degree)


    def __str__(self):
        monos = [self.display_monomial(k, coeff) for k, coeff in enumerate(self) if coeff != 0]
        return " + ".join(monos)


    def __call__(self, point):
        '''
        Evaluates the polynomial at the specified input
        '''
        return sum([co*point**i for i, co in enumerate(self)])


    def match_length(self, p):
        '''
        Adding polynomials requires them to be same length
        This method returns the same polynomial with additional zeros added to the end
        to match the length of the polynomial p
        '''
        added_zeros = [0 for i in range(len(p) - len(self))]
        return Polynomial(list(self) + added_zeros)

    def isnumber(self, p):
        from numbers import Number
        return isinstance(p, Number)
                    
                    
    def __add__(self, p):
        from numbers import Number
        # First, if we add a number rather than another polynomial,
        # we just return a polynomial whose constant (0th) term has
        # p added
        if self.isnumber(p):
            return self + Polynomial(p)

        polys = [self, p]
        polys.sort(key=len)
        matched_length = polys[0].match_length(polys[1])
        return Polynomial([co[0] + co[1] for co in zip(polys[1], matched_length)])

    def __neg__(self):
        return Polynomial([-a for a in self])

    def __sub__(self, p):
        return self + (-p)

    def same_sum_pairs(self, i, j): 
        '''
        Generates tuples containing pairs of powers
        having the same sum, upto required degree
        '''
        from itertools import product
        for k in range(i + j - 1): # need to -1 here because polynomial degree = highest power =
            yield (pair for pair in product(range(i), range(j)) if pair[0] + pair[1] == k)

    def __mul__(self,p):
        # Multiplication by a number just multiplies each
        # term by the number
        if self.isnumber(p):
            return self * Polynomial(p)
        deg1, deg2 = len(self), len(p)
        terms = [[self[i]*p[j] for i, j in degree_pairs] # list comprehensions within list comprehensions
                for degree_pairs in self.same_sum_pairs(deg1, deg2)] # never go deeper!
        return Polynomial([sum(term) for term in terms])

    def __radd__(self, p):
        return self + p 

    def __rmul__(self, p):
        return self * p

In [68]:
polyA = Polynomial(44, 23, 42)
polyB = Polynomial(3, 6)

In [69]:
print(polyA + polyB)

47 + 29x + 42x^2


In [70]:
print(polyA*polyB)

132 + 333x + 264x^2 + 252x^3


In [71]:
print(polyB*2)

6 + 12x


In [72]:
print(2*polyB)

6 + 12x


Cool. All working! The last thing is that derivative. This is quite straightforward. Using a list comprehension, we multiply each coefficient by its corresponding degree. Then to reduce the degree by 1, we output a new polynomial but omitting the first element of the list. So $3 + 3x + 4x^2$ first becomes $0 + 3x + 8x^2$. Then we omit the 0, and pass the $3$ and $8$ to the polynomial class to become $3 + 8x$, which is the correct derivative:

In [ ]:
    def derivative(self):
        coefficients = [i*coef for i, coef in enumerate(self)]
        return Polynomial(coefficients[1:])

The finished product:

In [73]:
class Polynomial(list):
    '''
    Polynomial class, essentially a list of coefficients with
    a nice string representation, and correctly behaving addition,
    multiplication, etc.
    '''
    def __init__(self, *coeffs):
        from collections.abc import Sequence
        if isinstance(coeffs[0], Sequence):
            coeffs = coeffs[0]
        list.__init__(self, coeffs)
        self.degree = self.find_degree()

    def find_degree(self):
        '''
        Find the degree of the polynomial.
        This is not necessarily the length of the polynomial
        as a list, since there may be redundant 0 terms at the end
        '''
        for i, coefficient in enumerate(self):
            if coefficient != 0:
                degree = i
        return degree

    def display_monomial(self, degree, coefficient):
        '''
        Correctly give a string representation of the given monomial
        '''
        if degree == 0:
            return str(coefficient)
        if degree == 1 and coefficient == 1:
            return self.symbol
        elif degree == 1 and coefficient != 1:
            return "{}{}".format(coefficient, "x")
        elif coefficient == 1:
            return "{}^{}".format("x", degree)
        else:
            return "{}{}^{}".format(coefficient, "x", degree)


    def __str__(self):
        monos = [self.display_monomial(k, coeff) for k, coeff in enumerate(self) if coeff != 0]
        return " + ".join(monos)


    def __call__(self, point):
        '''
        Evaluates the polynomial at the specified input
        '''
        return sum([co*point**i for i, co in enumerate(self)])


    def match_length(self, p):
        '''
        Adding polynomials requires them to be same length
        This method returns the same polynomial with additional zeros added to the end
        to match the length of the polynomial p
        '''
        added_zeros = [0 for i in range(len(p) - len(self))]
        return Polynomial(list(self) + added_zeros)

    def isnumber(self, p):
        from numbers import Number
        return isinstance(p, Number)
                    
                    
    def __add__(self, p):
        from numbers import Number
        # First, if we add a number rather than another polynomial,
        # we just return a polynomial whose constant (0th) term has
        # p added
        if self.isnumber(p):
            return self + Polynomial(p)

        polys = [self, p]
        polys.sort(key=len)
        matched_length = polys[0].match_length(polys[1])
        return Polynomial([co[0] + co[1] for co in zip(polys[1], matched_length)])

    def __neg__(self):
        return Polynomial([-a for a in self])

    def __sub__(self, p):
        return self + (-p)

    def same_sum_pairs(self, i, j): 
        '''
        Generates tuples containing pairs of powers
        having the same sum, upto required degree
        '''
        from itertools import product
        for k in range(i + j - 1): # need to -1 here because polynomial degree = highest power =
            yield (pair for pair in product(range(i), range(j)) if pair[0] + pair[1] == k)

    def __mul__(self,p):
        # Multiplication by a number just multiplies each
        # term by the number
        if self.isnumber(p):
            return self * Polynomial(p)
        deg1, deg2 = len(self), len(p)
        terms = [[self[i]*p[j] for i, j in degree_pairs] # list comprehensions within list comprehensions
                for degree_pairs in self.same_sum_pairs(deg1, deg2)] # never go deeper!
        return Polynomial([sum(term) for term in terms])

    def __radd__(self, p):
        return self + p 

    def __rmul__(self, p):
        return self * p

    def derivative(self):
        coefficients = [i*coef for i, coef in enumerate(self)]
        return Polynomial(coefficients[1:])

Let's test the derivative feature:

In [74]:
p = Polynomial(1, 2, 3, 4)
q = Polynomial([6, 3, 2, 1]) # remember: our poly doesn't care if it receives a list

In [79]:
print(p.derivative())

2 + 6x + 12x^2


In [80]:
# second derivatives
print(p.derivative().derivative())

6 + 24x


To make this nicer, we could create an alias for the derivative like this:

In [77]:
D = lambda p : p.derivative()

In [81]:
print(D(p))

2 + 6x + 12x^2


In [84]:
print(q*p + D(D(q)))

10 + 21x + 26x^2 + 38x^3 + 20x^4 + 11x^5 + 4x^6


See how easy the polynomial arithmetic is now we've made a class that can do it all using the normal arithmetic operations?

The derivative even obeys the Leibniz rule:

In [83]:
print(D(p*q) == D(p)*q + p*D(q))

True


As we have mentioned, there are many more magic methods. There's a pretty comprehensive list here http://www.diveintopython3.net/special-method-names.html, so check it out if you want your class to have a certain behaviour.

## Exercises

1. (Mathematical) Create a matrix class which can perform matrix multiplication and addition. You'll need to check if the two matrices have the correct dimensions to be added and multiplied!
2. Create a subclass of <code>list</code> for which the <code>.pop()</code> method returns and deletes the maximally valued element of the list, rather than the last element of the list.
3. In your Python installation, find the <code>collections</code> library. In the <code>\__init\__.py</code> file, find the <code>Counter</code> object, and look at the magic methods used to give it some of its extra behaviours. Try to unerstand at least the <code>\__add\__()</code> method.